# Understanding Memory in LLMs

In the previous Notebooks, we successfully explored how OpenAI models can enhance the results from Azure AI Search queries. 

However, we have yet to discover how to engage in a conversation with the LLM. With [Microsoft Copilot](http://chat.bing.com/), for example, this is possible, as it can understand and reference the previous responses.

There is a common misconception that LLMs (Large Language Models) have memory. This is not true. While they possess knowledge, they do not retain information from previous questions asked to them.

In this Notebook, our goal is to illustrate how we can effectively "endow the LLM with memory" by employing prompts and context.

In [1]:
import os
import random
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from typing import List

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from common.utils import CustomAzureSearchRetriever, get_answer
from common.prompts import DOCSEARCH_PROMPT_TEXT

from dotenv import load_dotenv
load_dotenv("credentials.env")

import logging

# Get the root logger
logger = logging.getLogger()
# Set the logging level to a higher level to ignore INFO messages
logger.setLevel(logging.WARNING)

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Let's start with the basics
Let's use a very simple example to see if the GPT model of Azure OpenAI have memory. We again will be using langchain to simplify our code 

In [3]:
QUESTION = "tell me chinese medicines that help fight covid-19"
FOLLOW_UP_QUESTION = "What was my prior question?"

In [4]:
COMPLETION_TOKENS = 1000
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

In [5]:
# We create a very simple prompt template, just the question as is:
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that give thorough responses to users."),
    ("user", "{input}")
])

In [6]:
# Let's see what the GPT model responds
chain = prompt | llm | output_parser
response_to_initial_question = chain.invoke({"input": QUESTION})
display(Markdown(response_to_initial_question))

Chinese medicine, which includes traditional practices such as herbal remedies, acupuncture, and other therapies, has been used in China as part of the integrated approach to manage COVID-19. It is important to note that while some traditional Chinese medicines (TCM) have been used in conjunction with conventional treatments, they should not replace medical advice or treatments prescribed by healthcare professionals. Here are some TCM formulations that have been used in China to support COVID-19 management:

1. **Lianhua Qingwen Capsule**: This is a well-known TCM formulation that contains a combination of herbs. It is traditionally used to relieve symptoms associated with viral respiratory infections, such as fever, cough, and fatigue.

2. **Jinhua Qinggan Granule**: This formulation is used to relieve symptoms of influenza-like illnesses. It contains several herbs that are believed to help clear heat and detoxify the lungs.

3. **Shufeng Jiedu Capsule**: This is another herbal formulation used to address symptoms of upper respiratory tract infections. It is thought to help dispel wind, clear heat, and relieve toxicity.

4. **Huoxiang Zhengqi Capsule**: Traditionally used to treat gastrointestinal symptoms, this formulation is sometimes used to address digestive issues that can occur with viral infections.

5. **Qingfei Paidu Decoction**: This is a classical TCM prescription that has been widely used in China during the COVID-19 pandemic. It is a combination of several herbal formulas and is believed to have a broad-spectrum antiviral effect.

6. **Xuanfei Baidu Granule**: Developed during the COVID-19 pandemic, this formulation is used to clear heat and remove toxins, supporting respiratory health.

7. **Pneumonia No. 1, 2, and 3 Decoctions**: These are specific formulations developed during the pandemic, used in some regions of China to treat COVID-19 symptoms.

While there are studies and anecdotal reports suggesting that these TCM formulations may help alleviate symptoms or support recovery, robust clinical trials are necessary to validate their efficacy and safety. Additionally, TCM should be used under the guidance of a qualified practitioner, especially when dealing with a complex condition like COVID-19.

It's crucial to follow public health guidelines and use evidence-based treatments for COVID-19. Vaccination, antiviral medications, and supportive care remain the primary strategies for managing COVID-19. Always consult with healthcare professionals before starting any new treatment.

In [7]:
#Now let's ask a follow up question
printmd(chain.invoke({"input": FOLLOW_UP_QUESTION}))

I'm sorry, but I don't have access to previous interactions or any personal data. Each session with me is stateless, meaning I don't retain information from past conversations. If you have a specific question or need information on a topic, feel free to ask, and I'll do my best to help!

As you can see, it doesn't remember what it just responded, sometimes it responds based only on the system prompt, or just randomly. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [8]:
hist_prompt = ChatPromptTemplate.from_template(
"""
    {history}
    Human: {question}
    AI:
"""
)
chain = hist_prompt | llm | output_parser

In [9]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response_to_initial_question)

In [10]:
printmd(chain.invoke({"history":Conversation_history, "question": FOLLOW_UP_QUESTION}))

Your prior question was: "tell me chinese medicines that help fight covid-19"

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

From Langchain website:
    
A memory system needs to support two basic actions: reading and writing. Recall that every chain defines some core execution logic that expects certain inputs. Some of these inputs come directly from the user, but some of these inputs can come from memory. A chain will interact with its memory system twice in a given run.

    AFTER receiving the initial user inputs but BEFORE executing the core logic, a chain will READ from its memory system and augment the user inputs.
    AFTER executing the core logic but BEFORE returning the answer, a chain will WRITE the inputs and outputs of the current run to memory, so that they can be referred to in future runs.
    
So this process adds delays to the response, but it is a necessary delay :)

![image](./images/memory_diagram.png)

In [11]:
index1_name = "srch-index-files"
index2_name = "srch-index-csv"
index3_name = "srch-index-books"
indexes = [index1_name, index2_name, index3_name]

In [12]:
# Initialize our custom retriever 
retriever = CustomAzureSearchRetriever(indexes=indexes, topK=50, reranker_threshold=1)

**Prompt Template Definition**

If you check closely below, there is an optional variable in the `DOCSEARCH_PROMPT` called `history`. It is basically a placeholder were we will inject the conversation in the prompt so the LLM is aware of it before it answers.


In [13]:

DOCSEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", DOCSEARCH_PROMPT_TEXT + "\n\nCONTEXT:\n{context}\n\n"),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{question}"),
    ]
)


**Now let's add memory to it:**

In [14]:
store = {} # Our first memory will be a dictionary in memory

# We have to define a custom function that takes a session_id and looks somewhere
# (in this case in a dictionary in memory) for the conversation
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [15]:
# We use our original chain with the retriever but removing the StrOutputParser
chain = (
    {
        "context": itemgetter("question") | retriever, 
        "question": itemgetter("question"),
        "history": itemgetter("history")
    }
    | DOCSEARCH_PROMPT
    | llm
)

## Then we pass the above chain to another chain that adds memory to it

output_parser = StrOutputParser()

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
) | output_parser

In [16]:
# This is where we configure the session id
config={"configurable": {"session_id": "abc123"}}

In [17]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

Empty Search Response


The tools did not provide relevant information. I cannot answer this from prior knowledge.

In [18]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Your prior question was: "tell me chinese medicines that help fight covid-19"

In [19]:
# Remembers
printmd(chain_with_history.invoke({"question": "Thank you! Good bye"},config=config))

You're welcome! Goodbye!

## Using CosmosDB as persistent memory

Previously, we  added local RAM memory to our chatbot. However, it is not persistent, it gets deleted once the app user's session is terminated. It is necessary then to use a Database for persistent storage of each of the  user conversations, not only for Analytics and Auditing, but also if we wish to provide recommendations in the future. 

In the next notebook we are going to explain how to use an external Database (CosmosDB) to keep the state of the conversation.

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We added persitent memory using local RAM.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, many times it searches for similar docs everytime, regardless of the input. This doesn't seem efficient, but regardless, we are very close to finish our first RAG talk-to-your-data bot.

Note:The use of `RunnableWithMessageHistory` in this notebook is for example purposes. We will see later (on the next notebooks), that we recomend the use of memory state and graphs in order to inject memory into an bot. 

# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

In the next notebook 6, we are going to build our first RAG bot. In order to do this we will introduce the concept of Agents.